In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.utils import shuffle
import os
from PIL import Image  
import time

import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Flatten, Dense, Conv2D, Dropout, MaxPool2D
from tensorflow.keras import optimizers
from keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.losses import CategoricalCrossentropy

%matplotlib inline

In [3]:
def load_data_numpy(dir):
    data_dir = os.path.join(dir, "data.npy") 
    label_dir = os.path.join(dir, "label.npy") 
    return np.load(data_dir), np.load(label_dir)

In [ ]:
def load_data_image(dir):
    LABEL_LIST = [10, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    
    X = []
    y_onehot = np.zeros((60000, 10))
    
    for num in range(10):
        sub_dir = os.path.join(dir, str(LABEL_LIST[num]))
        for img_dir in os.listdir(sub_dir):
            img = Image.open(os.path.join(sub_dir, img_dir))
            array_img = np.array(img)
            X.append(np.array(img))
        
        y_onehot[6000*(num):6000*(num+1), num] = 1
    
        # Show example of number
        # img.show(title=str(num+1))
        plt.figure()
        plt.title(str(LABEL_LIST[num]))
        plt.imshow(array_img)
        plt.show()
    
    X = np.reshape(np.array(X), (60000, 28, 28, 1))
    X_shuffle, y_shuffle = shuffle(X, y_onehot, random_state=1)
    
    # np.save('data.npy', X_shuffle)
    # np.save('label.npy', y_shuffle)
    
    return X_shuffle, y_shuffle

In [4]:
drive_folder = "/content/drive/My Drive/Colab Notebooks/training_data" 
X, y = load_data_numpy(drive_folder)
#my_folder = "training_data_image"
#X, y = load_data_image(my_folder)

# Splitting dataset

In [ ]:
X_train = X[:50000, :, :] 
y_train = y[:50000, :] 

X_small = X_train[:1000, :, :]
y_small = y[:1000, :] 

X_test = X[50000:, :, :] 
y_test = y[50000:, :] 

In [ ]:
model_ann = Sequential([
    Flatten(input_shape=(28,28)),
    Dense(500, activation='relu'),
    Dense(100, activation='relu'),
    Dense(10, activation='softmax')
])
opt_ann = keras.optimizers.Adam(1e-3)
model_ann.compile(optimizer=opt_ann, loss=CategoricalCrossentropy(), metrics=['accuracy'])
model_ann.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_17 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 500)               392500    
_________________________________________________________________
dense_47 (Dense)             (None, 100)               50100     
_________________________________________________________________
dense_48 (Dense)             (None, 10)                1010      
Total params: 443,610
Trainable params: 443,610
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_ann.fit(X_small, y_small, epochs=50, verbose=2)
test_loss, tess_acc = model_ann.evaluate(X_test, y_test)
print("Loss:", test_loss, "Accuracy:", tess_acc)

Epoch 1/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 2/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 3/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 4/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 5/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 6/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 7/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 8/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 9/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 10/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 11/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 12/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 13/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 14/50
32/32 - 0s - loss: 2.3009 - accuracy: 0.1150
Epoch 15/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 16/50
32/32 - 0s - loss: 2.3008 - accuracy: 0.1150
Epoch 17/50
32/32 - 0s - loss: 2.3009 - accuracy: 0.1150
Epoch 18/50
32/32 - 0s - loss: 2.3008 - 

# CNN

In [7]:
# BatchNormalization()
# MaxPooling2D(strides=2)
# momentum 
# activity_regularizer=l1(1e3)

model_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    #Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    #Dropout(0.25),
    
    Flatten(),
    Dense(512, activation='relu'),
    #Dropout(0.5),
    Dense(10, activation='softmax', activity_regularizer=l2(1e-4))
])
opt_cnn = optimizers.SGD(1e-4)
# default learning_rate = 1e-3, good learning_rate = 1e-4
model_cnn.compile(optimizer=opt_cnn, loss=CategoricalCrossentropy(), metrics=['accuracy'])
model_cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)             

In [8]:
# model_cnn.fit(X_small, y_small, validation_data=(X_test,y_test), epochs=100, verbose=2)
# test_loss, tess_acc = model_cnn.evaluate(X_test, y_test)
# print("Loss:", test_loss, "Accuracy:", tess_acc)

model_cnn.fit(X, y, validation_split=0.2, epochs=300, verbose=2)  # epochs=150-200 is good

Epoch 1/300
1500/1500 - 5s - loss: 2.3619 - accuracy: 0.2461 - val_loss: 1.8287 - val_accuracy: 0.3569
Epoch 2/300
1500/1500 - 5s - loss: 1.5895 - accuracy: 0.4351 - val_loss: 1.3955 - val_accuracy: 0.5044
Epoch 3/300
1500/1500 - 5s - loss: 1.2334 - accuracy: 0.5586 - val_loss: 1.1312 - val_accuracy: 0.5922
Epoch 4/300
1500/1500 - 5s - loss: 1.0035 - accuracy: 0.6395 - val_loss: 0.9666 - val_accuracy: 0.6534
Epoch 5/300
1500/1500 - 5s - loss: 0.8485 - accuracy: 0.6967 - val_loss: 0.7966 - val_accuracy: 0.7168
Epoch 6/300
1500/1500 - 5s - loss: 0.7310 - accuracy: 0.7402 - val_loss: 0.7185 - val_accuracy: 0.7415
Epoch 7/300
1500/1500 - 5s - loss: 0.6361 - accuracy: 0.7739 - val_loss: 0.6273 - val_accuracy: 0.7767
Epoch 8/300
1500/1500 - 5s - loss: 0.5612 - accuracy: 0.8011 - val_loss: 0.6046 - val_accuracy: 0.7865
Epoch 9/300
1500/1500 - 5s - loss: 0.5010 - accuracy: 0.8248 - val_loss: 0.5715 - val_accuracy: 0.8038
Epoch 10/300
1500/1500 - 5s - loss: 0.4534 - accuracy: 0.8403 - val_loss:

Training accuracy reach 100% after 150 epoches with loss at 0.005.
Validation accuracy is platue at 0.96 after 100 epoches with loss at 0.13 and then loss increase back to 0.15. 

# Saving model

In [ ]:
# import h5py
# model_cnn.save("cnn_model.h5")

# Loading model to run on test set

In [ ]:
from tensorflow.keras.models import load_model
new_model = load_model("cnn_model.h5")
new_model.summary()

In [ ]:
# test = 
# new_model.fit(test)